In [0]:
  ! pip install mtcnn

In [0]:
from google.colab import drive
import os
import tarfile
from keras.preprocessing import image as kimage
import matplotlib.pyplot as plt
import cv2
import numpy as np
import mtcnn #face detector
from PIL import Image
from PIL import ImageOps #flip photo
from tqdm import tqdm


In [0]:
drive.mount('/content/drive', force_remount=True)

In [0]:
tar= tarfile.open('/content/drive/My Drive/DSIM PROJECT/Face_Dataset.tar')
tar.extractall()
tar.close()

In [0]:
base_path = '/content/Face_Dataset/'
persone = os.listdir(base_path)
faces_path = '/content/drive/My Drive/DSIM PROJECT/Face_Detected_Dataset/'

In [0]:
def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    image1 = np.array(image1, dtype = np.float64)
    random_bright = .5+np.random.uniform()
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1[:,:,2][image1[:,:,2]>255]  = 255
    image1 = np.array(image1, dtype = np.uint8)
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

In [0]:
def extract_face(filename, required_size=(256, 256)):
  filename = Image.fromarray(filename)
  foto = filename.convert('RGB')
  # convert to array
  pixels = np.asarray(foto)
  # create the detector, using default weights
  detector = mtcnn.MTCNN()
  # detect faces in the image
  results = detector.detect_faces(pixels)
  if results != [] :
     # extract the bounding box from the first face
     x1, y1, width, height = results[0]['box']
     # bug fix
     x1, y1 = abs(x1), abs(y1)
     x2, y2 = x1 + width, y1 + height
     # extract the face
     face = pixels[y1:y2, x1:x2]
     # resize pixels to the model size
     image = Image.fromarray(face)
     image = image.resize(required_size)
     face_array = np.asarray(image)
  else:
    face_array = pixels
  return face_array

In [0]:
for persona in tqdm(persone) :
  persona_path = base_path + persona
  path2 = faces_path + persona
  for fi, f in tqdm(enumerate(sorted(os.listdir(persona_path)))) : 
    current_path = persona_path + '/' + f
    img = cv2.imread(current_path)
    aug = augment_brightness_camera_images(img)
    flip = cv2.flip(img, 1)
    face = extract_face(img)
    face_aug = extract_face(aug)
    face_flip = extract_face(flip)     

    cv2.imwrite(path2 + '/' + f , face)
    cv2.imwrite(path2 + '/bright' + f , face_aug)
    cv2.imwrite(path2 + '/flip' + f , face_flip)
